In [560]:
import re
import os

import pandas as pd

In [561]:
directory_path = '../../data/training-data/train-case-2/labeled/'

dataframes = []

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory_path, filename))
        dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

In [562]:
df = combined_df

In [563]:
df.columns = ["channel", "txt", "date", "category"]

In [564]:
df = df.dropna().copy()

In [565]:
df.head()

,channel,txt,date,category
0,Всевидящее ОКО 🇺🇦 | Новини Україна,**Сегодня отмечается международная неделя Клоу...,2023-08-01,Other
1,Всевидящее ОКО 🇺🇦 | Новини Україна,**Бойцы ВС РФ прямо сейчас отражают атаку бесп...,2023-08-04,Crisis
2,Всевидящее ОКО 🇺🇦 | Новини Україна,Історія з офісами телефонних шахраїв заграла я...,2023-08-09,Corruption
3,Всевидящее ОКО 🇺🇦 | Новини Україна,**Самый популярный блогер в мире MrBeast выпус...,2023-08-19,Political
4,Всевидящее ОКО 🇺🇦 | Новини Україна,**🔥**** Точно в цель: впечатляющая эффективнос...,2023-08-26,Crisis


In [566]:
df["category"] = df["category"].replace("Shelling", "Crisis")

In [567]:
df[df["category"] == "Shelling"].shape

(0, 4)

In [568]:
df['txt'] = df['txt'].replace('\*\*|__|~~', '', regex=True)
df['txt'] = df['txt'].replace(r'["“”«»„"]\s*["“”«»„"]', '', regex=True)
df['txt'] = df['txt'].replace('""', '')

In [569]:
df['txt'] = df['txt'].replace(
    '(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])',
    ' ',
    regex=True,
)

In [570]:
df['txt'] = df['txt'].replace(r'\n|\r|\t', ' ', regex=True)

In [571]:
df['txt'] = df['txt'].replace(r'\[|\]', '', regex=True)

In [572]:
words_to_remove = [
    "Покупаем контент", "ПОДПИСАТЬСЯ", "𝔼ℂ𝕆ℕ𝕆𝕄𝕀𝕂𝔸", "Подпишись — RT на русском", "Подпишись на канал «Специально для RT»",
    "підписатися", "Подписаться на канал УНИАН", "Купуємо контент"
]

In [573]:
for word in words_to_remove:
    df['txt'] = df['txt'].str.replace(word, '')

In [574]:
df["txt"] = df['txt'].replace(r'\(\s*\)', '', regex=True)

In [575]:
df['txt'] = df['txt'].replace(r'\s+', ' ', regex=True)
df['txt'] = df['txt'].str.strip()

df['txt'] = df['txt'].str.lower()

In [576]:
def remove_emojis(string):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', string)

In [577]:
df['txt'] = df['txt'].apply(remove_emojis)

In [578]:
df['txt'] = df['txt'].str.replace('""', '')

In [579]:
df.to_csv("../../data/training-data/train-case-2/final-preprocessed-dataset.csv", index=False)